In [1]:
import numpy as np

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json
from geopy.geocoders import Nominatim

from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium

import requests
from bs4 import BeautifulSoup

import geocoder
import time

from re import sub
from decimal import Decimal

print('Libraries imported.')

Libraries imported.


In [2]:
url = 'https://www.seattlemet.com/home-and-real-estate/2019/03/the-top-12-neighborhoods-in-seattle-2019-edition'
data  = requests.get(url).text
soup = BeautifulSoup(data, 'html5lib')

neighborhoods = []
for neigh in soup.find_all('h3')[1:]:
    if '(' in neigh.get_text():
        neighb = neigh.get_text().split(' (')[0]
    else:
        neighb = neigh.get_text()
    if neighb in ["Seattle's Hottest Neighborhoods by the Numbers", 'Channels', 'About Us', 'Subscriptions', 'Other SagaCity Media']:
        continue
    neighborhoods.append(neighb)

average_rent_usd = []
for each_tag in soup.find_all('blockquote'):
    for data_entry in each_tag.get_text().split('\xa0 •\xa0'):
        if 'Rent' in data_entry:
            average_rent_usd.append(Decimal(sub(r'[^\d.]', '', data_entry.split('$')[1])))

print(neighborhoods)

['West Seattle', 'Beacon Hill', 'Ravenna', 'Columbia City', 'Ballard', 'Interbay', 'South Park', 'Central District', 'Georgetown', 'Lake City', 'Greenwood', 'Capitol Hill']


In [3]:
locator = Nominatim(user_agent='myGeocoder')

lat = []
lon = []
for address in neighborhoods:
    address = address + ', Seattle, WA'
    location = locator.geocode(address)
    lat.append(location.latitude)
    lon.append(location.longitude)

seattle_neigh = pd.DataFrame()
seattle_neigh['Neighborhood'] = neighborhoods
seattle_neigh['Average Rent (USD)'] = average_rent_usd
seattle_neigh['Latitude'] = lat
seattle_neigh['Longitude'] = lon
seattle_neigh.head()

,Neighborhood,Average Rent (USD),Latitude,Longitude
0,West Seattle,1832,47.570932,-122.386517
1,Beacon Hill,1634,47.579271,-122.311794
2,Ravenna,1598,47.675654,-122.297626
3,Columbia City,2272,47.557912,-122.285216
4,Ballard,2174,47.676507,-122.386223


In [4]:
seattle_neigh.to_csv('neighborhoods-data.csv')

In [5]:
address = 'Seattle, WA'

geolocator = Nominatim(user_agent="sea-surfer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
map_seattle = folium.Map(location=[latitude, longitude], zoom_start=11, tiles='CartoDB positron')

for lat, lng, neighborhood in zip(seattle_neigh['Latitude'], seattle_neigh['Longitude'], seattle_neigh['Neighborhood']):
    label = neighborhood
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_seattle)  
    
map_seattle

In [6]:
# Sorting neighborhoods with respect to their average rent
# seattle_neigh.sort_values('Average Rent (USD)', inplace=True)
# seattle_neigh

In [7]:
CLIENT_ID = 'MHHLLZOS144U44BZKQTGKNAM4PX2Z10NOKBOK5A2GRCRMKM4'
CLIENT_SECRET = 'FHXGOYHIHHRJDML5YBJX1GYL4OVJTQTW15JFKKT20RPTOREW'
VERSION = '20180605'
radius = 500
LIMIT = 100

In [8]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [9]:
seattle_venues = getNearbyVenues(seattle_neigh['Neighborhood'], seattle_neigh['Latitude'], seattle_neigh['Longitude'])
seattle_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,West Seattle,47.570932,-122.386517,Olympia Coffee,47.569384,-122.386682,Coffee Shop
1,West Seattle,47.570932,-122.386517,Meeples Games,47.570496,-122.387040,Gaming Cafe
2,West Seattle,47.570932,-122.386517,Prost - West Seattle,47.573494,-122.386959,Pub
3,West Seattle,47.570932,-122.386517,West Seattle Runner,47.570363,-122.387090,Sporting Goods Shop
4,West Seattle,47.570932,-122.386517,Spiro's Pizza & Pasta - West Seattle,47.573744,-122.386958,Pizza Place


In [10]:
seattle_venues.to_csv('venues-near-neighborhoods.csv')

In [11]:
seattle_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Ballard,11,11,11,11,11,11
Beacon Hill,18,18,18,18,18,18
Capitol Hill,56,56,56,56,56,56
Central District,18,18,18,18,18,18
Columbia City,35,35,35,35,35,35
Georgetown,34,34,34,34,34,34
Greenwood,44,44,44,44,44,44
Interbay,10,10,10,10,10,10
Lake City,36,36,36,36,36,36


In [12]:
print('There are {} uniques categories.'.format(len(seattle_venues['Venue Category'].unique())))

There are 128 uniques categories.


In [13]:
# one hot encoding
seattle_onehot = pd.get_dummies(seattle_venues[['Venue Category']], prefix="", prefix_sep="")

#seattle_onehot = seattle_onehot.drop(['Neighborhood'], axis=1)

# add neighborhood column back to dataframe
seattle_onehot['Neighborhood'] = seattle_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [seattle_onehot.columns[-1]] + list(seattle_onehot.columns[:-1])
seattle_onehot = seattle_onehot[fixed_columns]

seattle_onehot.head()

,Neighborhood,ATM,African Restaurant,American Restaurant,Arcade,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bakery,Bank,Bar,Baseball Field,Bed & Breakfast,Beer Bar,Beer Store,Bike Shop,Bookstore,Breakfast Spot,Brewery,Burger Joint,Bus Station,Bus Stop,Café,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comic Shop,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Dance Studio,Department Store,Design Studio,Diner,Dog Run,Electronics Store,Ethiopian Restaurant,Event Space,Farmers Market,Filipino Restaurant,Fish & Chips Shop,Fish Market,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gas Station,Gastropub,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Hawaiian Restaurant,Herbs & Spices Store,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Liquor Store,Lounge,Marijuana Dispensary,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Music Store,Music Venue,Nail Salon,New American Restaurant,Noodle House,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Pub,Record Shop,Rental Car Location,Restaurant,Salad Place,Salon / Barbershop,Salsa Club,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shipping Store,Smoke Shop,Soccer Field,Southern / Soul Food Restaurant,Spa,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,West Seattle,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,West Seattle,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,West Seattle,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,West Seattle,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,West Seattle,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [14]:
seattle_grouped = seattle_onehot.groupby('Neighborhood').mean().reset_index()
seattle_grouped

,Neighborhood,ATM,African Restaurant,American Restaurant,Arcade,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bakery,Bank,Bar,Baseball Field,Bed & Breakfast,Beer Bar,Beer Store,Bike Shop,Bookstore,Breakfast Spot,Brewery,Burger Joint,Bus Station,Bus Stop,Café,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comic Shop,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Dance Studio,Department Store,Design Studio,Diner,Dog Run,Electronics Store,Ethiopian Restaurant,Event Space,Farmers Market,Filipino Restaurant,Fish & Chips Shop,Fish Market,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gas Station,Gastropub,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Hawaiian Restaurant,Herbs & Spices Store,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Liquor Store,Lounge,Marijuana Dispensary,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Music Store,Music Venue,Nail Salon,New American Restaurant,Noodle House,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Pub,Record Shop,Rental Car Location,Restaurant,Salad Place,Salon / Barbershop,Salsa Club,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shipping Store,Smoke Shop,Soccer Field,Southern / Soul Food Restaurant,Spa,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,Ballard,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.090909,0.090909,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.090909,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.090909,0.090909,0.000000,0.000000,0.000000,0.000000,0.000000,0.090909,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.090909,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.090909,0.000000,0.000000,0.000000,0.000000,0.090909,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.090909,0.000000,0.000000,0.000000,0.000000,0.090909,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Beacon Hill,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.055556,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.055556,0.000000,0.000000,0.000000,0.000000,0.055556,0.000000,0.000000,0.111111,0.000000,0.000000,0.0,0.000000,0.055556,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556,0.000000,0.055556,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556,0.000000,0.000000,0.055556,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000

In [15]:
num_top_venues = 5

for hood in seattle_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = seattle_grouped[seattle_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Ballard----
                   venue  freq
0             Playground  0.09
1                   Park  0.09
2      French Restaurant  0.09
3  Sri Lankan Restaurant  0.09
4         Baseball Field  0.09


----Beacon Hill----
                      venue  freq
0                      Café  0.11
1               Gas Station  0.11
2               Yoga Studio  0.06
3  Mediterranean Restaurant  0.06
4               Pizza Place  0.06


----Capitol Hill----
                venue  freq
0         Coffee Shop  0.09
1        Cocktail Bar  0.07
2  Italian Restaurant  0.07
3     Thai Restaurant  0.07
4  Mexican Restaurant  0.05


----Central District----
                   venue  freq
0            Coffee Shop  0.17
1  Performing Arts Venue  0.11
2             Playground  0.06
3              BBQ Joint  0.06
4         Breakfast Spot  0.06


----Columbia City----
                venue  freq
0         Coffee Shop  0.09
1         Pizza Place  0.06
2                 Bar  0.06
3  African Restaurant  0.06
4   

In [16]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [17]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = seattle_grouped['Neighborhood']

for ind in np.arange(seattle_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(seattle_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Ballard,Playground,French Restaurant,Park,Soccer Field,Baseball Field,Bar,Sri Lankan Restaurant,Food Truck,Jewelry Store,Coffee Shop
1,Beacon Hill,Gas Station,Café,Yoga Studio,Bakery,Pizza Place,Grocery Store,Pub,Dance Studio,Mexican Restaurant,Brewery
2,Capitol Hill,Coffee Shop,Thai Restaurant,Italian Restaurant,Cocktail Bar,Mexican Restaurant,Yoga Studio,Japanese Restaurant,Bar,American Restaurant,Pet Store
3,Central District,Coffee Shop,Performing Arts Venue,BBQ Joint,Chinese Restaurant,Playground,Plaza,Salsa Club,Breakfast Spot,Ethiopian Restaurant,Fish & Chips Shop
4,Columbia City,Coffee Shop,African Restaurant,Ice Cream Shop,Bar,Pizza Place,Gymnastics Gym,Mexican Restaurant,Pub,Caribbean Restaurant,Park


In [18]:
# set number of clusters
kclusters = 4

seattle_grouped_clustering = seattle_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(seattle_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 1, 3, 3, 3, 3, 3, 0, 3, 1], dtype=int32)

In [19]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

seattle_merged = seattle_neigh

# merge seattle_grouped with seattle_data to add latitude/longitude for each neighborhood
seattle_merged = seattle_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

seattle_merged.head() # check the last columns!

,Neighborhood,Average Rent (USD),Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,West Seattle,1832,47.570932,-122.386517,3,Pizza Place,Sporting Goods Shop,Convenience Store,Pub,Cosmetics Shop,Music Store,Sandwich Place,Coffee Shop,Kids Store,Park
1,Beacon Hill,1634,47.579271,-122.311794,1,Gas Station,Café,Yoga Studio,Bakery,Pizza Place,Grocery Store,Pub,Dance Studio,Mexican Restaurant,Brewery
2,Ravenna,1598,47.675654,-122.297626,1,Mediterranean Restaurant,Yoga Studio,Sushi Restaurant,Pizza Place,Grocery Store,New American Restaurant,Creperie,Café,Bus Stop,Bus Station
3,Columbia City,2272,47.557912,-122.285216,3,Coffee Shop,African Restaurant,Ice Cream Shop,Bar,Pizza Place,Gymnastics Gym,Mexican Restaurant,Pub,Caribbean Restaurant,Park
4,Ballard,2174,47.676507,-122.386223,3,Playground,French Restaurant,Park,Soccer Field,Baseball Field,Bar,Sri Lankan Restaurant,Food Truck,Jewelry Store,Coffee Shop


In [20]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11, tiles = 'CartoDB positron')

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
seattle_merged = seattle_merged.dropna(axis=0)
for lat, lon, poi, cluster in zip(seattle_merged['Latitude'], seattle_merged['Longitude'], seattle_merged['Neighborhood'], seattle_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [21]:
# Sorting neighborhoods with respect to their average rent
seattle_merged.sort_values('Average Rent (USD)', inplace=True)
seattle_merged.to_csv('sorted_seattle_neighborhoods.csv')
seattle_merged

,Neighborhood,Average Rent (USD),Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Lake City,1379,47.719162,-122.295494,3,Pub,Bank,Thai Restaurant,Pizza Place,Mexican Restaurant,Gymnastics Gym,Noodle House,Breakfast Spot,Rental Car Location,Burger Joint
5,Interbay,1525,47.640742,-122.376339,0,Golf Course,Wine Shop,Grocery Store,Pet Store,Ice Cream Shop,Arts & Crafts Store,Electronics Store,Bike Shop,Department Store,Diner
2,Ravenna,1598,47.675654,-122.297626,1,Mediterranean Restaurant,Yoga Studio,Sushi Restaurant,Pizza Place,Grocery Store,New American Restaurant,Creperie,Café,Bus Stop,Bus Station
1,Beacon Hill,1634,47.579271,-122.311794,1,Gas Station,Café,Yoga Studio,Bakery,Pizza Place,Grocery Store,Pub,Dance Studio,Mexican Restaurant,Brewery
8,Georgetown,1800,47.548933,-122.330057,3,Coffee Shop,Food Truck,Bar,Japanese Restaurant,Sandwich Place,Furniture / Home Store,Fish Market,Hawaiian Restaurant,Design Studio,Construction & Landscaping
10,Greenwood,1820,47.690981,-122.354877,3,Coffee Shop,Mexican Restaurant,Bar,Pizza Place,Spa,Bookstore,Diner,Herbs & Spices Store,Mediterranean Restaurant,Indian Restaurant
0,West Seattle,1832,47.570932,-122.386517,3,Pizza Place,Sporting Goods Shop,Convenience Store,Pub,Cosmetics Shop,Music Store,Sandwich Place,Coffee Shop,Kids Store,Park
7,Central District,1948,47.603110,-122.308270,3,Coffee Shop,Performing Arts Venue,BBQ Joint,Chinese Restaurant,Playground,Plaza,Salsa Club,Breakfast Spot,Ethiopian Restaurant,Fish & Chips Shop
11,Capitol Hill,2070,47.623831,-122.318368,3,Coffee Shop,Thai Restaurant,Italian Restaurant,Cocktail Bar,Mexican Restaurant,Yoga Studio,Japanese Restaurant,Bar,American Restaurant,Pet Store
4,Ballard,2174,47.676507,-122.386223,3,Playground,French Restaurant,Park,Soccer Field,Baseball Field,Bar,Sri Lankan Restaurant,Food Truck,Jewelry Store,Coffee Shop
